In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from chromadb.utils import embedding_functions
import os
import chromadb
from chromadb.config import Settings

In [ ]:
client = chromadb.PersistentClient(path="chromadb/")

In [ ]:
client.count_collections()

# Split docs to chunk

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)

# Create LLM | Embedding

In [ ]:
embeddings = embedding_functions.DefaultEmbeddingFunction()

# Main 

In [ ]:
%%time
if __name__ == "__main__":

    # Iterate through PDF files in the articles directory
    articles_dir = "article/"
    ids = []
    pages = []
    
    for filename in os.listdir(articles_dir):
        if filename.endswith(".pdf"):
            file_path = os.path.join(articles_dir, filename)
            loader = PyPDFLoader(file_path)
            doc = loader.load()    
    
        # Split doc to chunks
        pages = text_splitter.split_documents(doc)
        ids = [str(i) for i in range(len(pages))]
        print(f"Generated {len(pages)} documents of {filename}.")
    
        # save doc to vector store
        collection = client.get_or_create_collection(f"{filename}")
        collection.add(
            documents = [page.page_content for page in pages],
            metadatas = [page.metadata for page in pages],
            ids = ids
        )
        
        